In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
from data_prep.input_formatting import extract_simfile_data, cnn_xy_split, filter_training_data
import os
import simfile
from simfile.notes import NoteData, Note, NoteType
from simfile.timing import TimingData, Beat
from simfile.timing.engine import TimingEngine
import librosa

In [2]:
df_list = []
for folder in os.listdir("pack_37_9ddada"):
    folder = os.path.join("pack_37_9ddada", folder)
    sm_path = ""
    ogg_path = ""
    if not os.path.isfile(folder):
        for file in os.listdir(folder):
            if file.endswith(".sm"):
                sm_path = file
            if file.endswith(".ogg"):
                ogg_path = file
        ogg_path = os.path.join(folder + "/", ogg_path)
        sm_path = os.path.join(folder + "/", sm_path)
        print(file)
        df_list.append(extract_simfile_data(ogg_path, sm_path, "dance-single", "Hard"))
            

Butterfly-bg.png
STRICTLY BUSINESS-bg.png
My Fire (UKS Remix).ogg
PARANOiA-jacket.png
Have You Never Been Mellow-jacket.png
KUNG FU FIGHTING-jacket.png
Little Bitch.ogg
TRIP MACHINE-bg.png
LET'S GET DOWN-bg.png
MAKE IT BETTER.png
That's The Way (I Like It).ogg


In [3]:
print(len(df_list[1:]))
train_list = df_list[1:]
test_song = df_list[0]

10


In [4]:
for i in range(len(train_list)):
    train_list[i] = filter_training_data(train_list[i])

training_set = pd.concat(train_list)

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution1D, MaxPooling1D
from keras.utils import np_utils

In [28]:
X_train = training_set.loc[:,training_set.columns.difference(["target"])].to_numpy()
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = training_set["target"].to_numpy()
y_train = np_utils.to_categorical(y_train)

In [7]:
X_train.shape

(3960, 12, 1)

In [8]:
model = Sequential()
model.add(Convolution1D(32, 8, activation='relu', input_shape=(12,1)))
print(model.output_shape)
model.add(Convolution1D(32, 3, activation='relu'))
print(model.output_shape)
model.add(MaxPooling1D(pool_size=2))
print(model.output_shape)
model.add(Dropout(0.25))
print(model.output_shape)

(None, 5, 32)
(None, 3, 32)
(None, 1, 32)
(None, 1, 32)


In [9]:

model.add(Flatten())
print(model.output_shape)
model.add(Dense(128, activation='relu'))
print(model.output_shape)
model.add(Dropout(0.25))
print(model.output_shape)
model.add(Dense(5, activation='softmax'))

(None, 32)
(None, 128)
(None, 128)


In [10]:

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:

model.fit(X_train, y_train, 
          batch_size=32, epochs=10, verbose=1)

Epoch 1/10


2022-11-25 21:50:59.564933: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


124/124 [==============================] - 0s 993us/step - loss: 1.3984 - accuracy: 0.4990
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 1.3008 - accuracy: 0.5000
Epoch 3/10
124/124 [==============================] - 0s 997us/step - loss: 1.2907 - accuracy: 0.5000
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 1.2805 - accuracy: 0.5000
Epoch 5/10
124/124 [==============================] - 0s 978us/step - loss: 1.2702 - accuracy: 0.5000
Epoch 6/10
124/124 [==============================] - 0s 997us/step - loss: 1.2585 - accuracy: 0.5000
Epoch 7/10
124/124 [==============================] - 0s 993us/step - loss: 1.2484 - accuracy: 0.5010
Epoch 8/10
124/124 [==============================] - 0s 990us/step - loss: 1.2368 - accuracy: 0.4992
Epoch 9/10
124/124 [==============================] - 0s 1ms/step - loss: 1.2313 - accuracy: 0.5025
Epoch 10/10
124/124 [==============================] - 0s 990us/step - loss: 1.2234 - accuracy: 0.5

In [12]:
test_df = test_song

X_test, y_test = cnn_xy_split(test_df)

model.evaluate(X_test, y_test, verbose=1)

127/127 [==============================] - 0s 463us/step - loss: 0.8540 - accuracy: 0.9328


[0.8539767861366272, 0.9328063130378723]

In [13]:
results = model.predict(X_test)

127/127 [==============================] - 0s 369us/step


In [15]:
int_results = []

for result in results:
    int_results.append(np.argmax(result))

In [16]:
print(len(int_results))
print(len(test_df.index))

4048
4048


In [18]:
run_count = 0
begin_run = True
mark_list = [0] * len(int_results)
for idx, item in enumerate(int_results):
    if idx > 0:
        if item != 4 and int_results[idx-1] == item:
            mark_list[idx] = 1


In [19]:
for idx, item in enumerate(mark_list):
    if item:
        int_results[idx] = 4


In [21]:
new_sm = open("test_song.sm", "x")

FileExistsError: [Errno 17] File exists: 'test_song.sm'

In [22]:
sound_file, sr = librosa.load("pack_37_9ddada/Butterfly/Butterfly.ogg")
bpm = librosa.beat.tempo(sound_file)

/var/folders/87/fylw6rqx7cz0vrk0z9cdsbhh0000gn/T/ipykernel_22466/1563841414.py:2: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 9.5100899e-05 9.9585202e-05
 1.0719002e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm = librosa.beat.tempo(sound_file)


In [23]:
print(bpm)

[135.99917763]


KNeighborsClassifier(n_neighbors=3)

In [25]:
input_filename = 'test_song.sm'
samps_per_sec = 1/sr
with simfile.mutate(
    input_filename,
    backup_filename=f'{input_filename}.old',
) as test_song:
    test_song.charts.append(simfile.sm.SMChart())
    test_song.charts[0].meter = 8
    test_song.charts[0].difficulty = "Hard"
    test_song.charts[0].stepstype = "dance-single"
    test_song.bpms = "0.000="+str(round(bpm[0],2))
    test_song.music = "Butterfly.ogg"
    timing_data = TimingData(test_song)
    engine = TimingEngine(timing_data)
    print(engine.beat_at(10.0))
    notes = []
    for idx, item in enumerate(int_results):
        if item != 4:
            notes.append(Note(beat=Beat(engine.beat_at(idx*512*samps_per_sec)), column=item, note_type=NoteType.TAP))
    notes = NoteData.from_notes(notes, columns=4)
    test_song.charts[0].notes = str(notes)


22.667


KNeighborsClassifier(n_neighbors=3)

In [26]:
butt = simfile.open("pack_37_9ddada/Butterfly/Butterfly.sm")

In [29]:
butt.bpms

'0.000=31.250,4.000=134.922'